In [2]:
import datetime
import ast
import json
from collections import Counter

import numpy as np
import pandas as pd
import string
import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor

#### Read Dataset

In [3]:
df = pd.read_csv('cars.csv')
df.head()

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,transmission,engine,vin,stock_number,vehicle_history,seller_name,price_history
0,https://www.cars.com/vehicledetail/5ee3ae04-69...,2018 Hyundai Santa Fe Sport 2.4L,"79,497 mi.","$17,899",Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,6-Speed Automatic,2.4L I4 16V GDI DOHC,5XYZT3LB3JG542718,13669,"{'Accidents or damage': 'None reported', '1-ow...",ETHIO Motors,"[('9/21/21', 'Listed', '$20,299'), ('2/04/22',..."
1,https://www.cars.com/vehicledetail/c99e5035-76...,2017 RAM 1500 Express,"80,326 mi.","$27,900",This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,3C6RR7KT1HG533082,KBB2955,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('3/15/22', 'Listed', '$30,985'), ('6/01/22',..."
2,https://www.cars.com/vehicledetail/4e79fa3a-ba...,2019 Nissan Kicks SR,"92,959 mi.","$18,950",This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,Automatic CVT,1.6L I4 16V MPFI DOHC,3N1CP5CU6KL510673,KBB3225,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('4/16/22', 'Listed', '$19,980'), ('6/01/22',..."
3,https://www.cars.com/vehicledetail/86a0d096-32...,2019 RAM 1500 Laramie,"59,176 mi.","$39,950",Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,1C6SRFDT5KN869809,KBB3246,{'Accidents or damage': 'At least 1 accident o...,Custom Cars West,"[('6/01/22', 'Listed', '$40,523'), ('7/14/22',..."
4,https://www.cars.com/vehicledetail/36efbef5-f2...,2015 Jeep Wrangler Unlimited Rubicon,"135,385 mi.","$26,950",This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,-,3.6L V6 24V MPFI DOHC,1C4BJWFG2FL530006,KBB3145,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('1/21/22', 'Listed', '$29,950'), ('3/22/22',..."


In [3]:
# Enclose link in quotes so they can be ctrl clicked in vs code
df['link'] = df['link'].apply(lambda x: f'"{x}"')
df.head()

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,transmission,engine,vin,stock_number,vehicle_history,seller_name,price_history
0,"""https://www.cars.com/vehicledetail/5ee3ae04-6...",2018 Hyundai Santa Fe Sport 2.4L,"79,497 mi.","$17,899",Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,6-Speed Automatic,2.4L I4 16V GDI DOHC,5XYZT3LB3JG542718,13669,"{'Accidents or damage': 'None reported', '1-ow...",ETHIO Motors,"[('9/21/21', 'Listed', '$20,299'), ('2/04/22',..."
1,"""https://www.cars.com/vehicledetail/c99e5035-7...",2017 RAM 1500 Express,"80,326 mi.","$27,900",This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,3C6RR7KT1HG533082,KBB2955,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('3/15/22', 'Listed', '$30,985'), ('6/01/22',..."
2,"""https://www.cars.com/vehicledetail/4e79fa3a-b...",2019 Nissan Kicks SR,"92,959 mi.","$18,950",This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,Automatic CVT,1.6L I4 16V MPFI DOHC,3N1CP5CU6KL510673,KBB3225,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('4/16/22', 'Listed', '$19,980'), ('6/01/22',..."
3,"""https://www.cars.com/vehicledetail/86a0d096-3...",2019 RAM 1500 Laramie,"59,176 mi.","$39,950",Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,1C6SRFDT5KN869809,KBB3246,{'Accidents or damage': 'At least 1 accident o...,Custom Cars West,"[('6/01/22', 'Listed', '$40,523'), ('7/14/22',..."
4,"""https://www.cars.com/vehicledetail/36efbef5-f...",2015 Jeep Wrangler Unlimited Rubicon,"135,385 mi.","$26,950",This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,-,3.6L V6 24V MPFI DOHC,1C4BJWFG2FL530006,KBB3145,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('1/21/22', 'Listed', '$29,950'), ('3/22/22',..."


## Split vehicle_history

In [4]:
def extract_dict(s):
    try:
        # Convert the string to a dictionary using literal_eval
        return ast.literal_eval(s)
    except (SyntaxError, ValueError):
        # Handle cases where literal_eval fails, e.g., malformed strings
        return {}

# First convert dtype to str from object
df['vehicle_history'] = df['vehicle_history'].astype(str)
# Apply the extract_dict fcn
df['vehicle_history'] = df['vehicle_history'].apply(extract_dict)
# normalize and join
norm = pd.json_normalize(df['vehicle_history'])
df = pd.concat([df, norm], axis= 1)

# Display
selected_columns = ['Accidents or damage', '1-owner vehicle', 'Personal use only', 'Open recall', 'Clean title', 'vehicle_history']
df[selected_columns].head()

,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title,vehicle_history
0,None reported,Yes,No,At least 1 open recall reported,NaN,"{'Accidents or damage': 'None reported', '1-ow..."
1,None reported,No,Yes,NaN,NaN,"{'Accidents or damage': 'None reported', '1-ow..."
2,None reported,No,Yes,NaN,NaN,"{'Accidents or damage': 'None reported', '1-ow..."
3,At least 1 accident or damage reported,No,No,At least 1 open recall reported,NaN,{'Accidents or damage': 'At least 1 accident o...
4,None reported,No,Yes,At least 1 open recall reported,NaN,"{'Accidents or damage': 'None reported', '1-ow..."


## Split listing_title

In [5]:
df[['year','make','model']] = df.listing_title.str.split(' ',2, expand=True)
selected_columns = ['year','make','model', 'listing_title']
df[selected_columns].head()

,year,make,model,listing_title
0,2018,Hyundai,Santa Fe Sport 2.4L,2018 Hyundai Santa Fe Sport 2.4L
1,2017,RAM,1500 Express,2017 RAM 1500 Express
2,2019,Nissan,Kicks SR,2019 Nissan Kicks SR
3,2019,RAM,1500 Laramie,2019 RAM 1500 Laramie
4,2015,Jeep,Wrangler Unlimited Rubicon,2015 Jeep Wrangler Unlimited Rubicon


## Split price_history
### listed_date, price_change, price_change_percentage

In [6]:
# Convert string representation of lists to actual lists
df['price_history'] = df['price_history'].apply(ast.literal_eval)

# Extract the oldest date, total price change, and price change percentage
df['listed_date'] = df['price_history'].apply(lambda x: min([pd.to_datetime(item[0], format='%m/%d/%y')for item in x], default=None))

# Calculate total price change and price change percentage
def calculate_changes(prices):
    if not prices:
        return None, None
    prices = [(pd.to_datetime(date, format='%m/%d/%y'), status, int(price.replace('$', '').replace(',', ''))) for date, status, price in prices]
    total_price_change = prices[-1][2] - prices[0][2]
    price_change_percentage = (total_price_change / prices[0][2]) * 100
    return total_price_change, price_change_percentage

df['price_change'], df['price_change_percentage'] = zip(*df['price_history'].apply(calculate_changes))

# Display
selected_columns = ['listed_date', 'price_change', 'price_change_percentage', 'price_history']
df[selected_columns].head()

,listed_date,price_change,price_change_percentage,price_history
0,2021-09-21,-2400.0,-11.823243,"[(9/21/21, Listed, $20,299), (2/04/22, +$201, ..."
1,2022-03-15,-3085.0,-9.956431,"[(3/15/22, Listed, $30,985), (6/01/22, -$200, ..."
2,2022-04-16,-1030.0,-5.155155,"[(4/16/22, Listed, $19,980), (6/01/22, -$200, ..."
3,2022-06-01,-573.0,-1.414012,"[(6/01/22, Listed, $40,523), (7/14/22, -$573, ..."
4,2022-01-21,-3000.0,-10.016694,"[(1/21/22, Listed, $29,950), (3/22/22, -$2,000..."


## Remove dollar sign and commas from primary price

In [7]:
df['primary_price'] = df['primary_price'].replace('[\$,]', '', regex=True).astype(int)
df[['primary_price']].head()

,primary_price
0,17899
1,27900
2,18950
3,39950
4,26950


## Remove “ mi.” abbreviation and commas from listing_mileage 

In [8]:
df['listing_mileage'] = df['listing_mileage'].str.replace(',', '', regex=True).str.replace(' mi.', '', regex=True).astype(int)
df[['listing_mileage']].head()

,listing_mileage
0,79497
1,80326
2,92959
3,59176
4,135385


## Check unique values to see what we're working with: 
### engine, transmission, drivetrain. 

In [9]:
unique_engines = sorted(df['engine'].unique())
unique_transmissions = sorted(df['transmission'].unique())
unique_drivetrains = sorted(df['drivetrain'].unique())
unique_fuel_types = sorted(df['fuel_type'].unique())

def print_unqiues():
    # Print the unique values
    print("Sorted Unique Engines:")
    for engine in unique_engines:
        print(engine)
    # Want to isolate: 
    # cylinders
    # liters
    # boosted ("Turbo"/"Supercharge") T/F

    print("\nSorted Unique Transmissions:")
    for transmission in unique_transmissions:
        print(transmission)
    # Not worth the effort to differentiate automatic versus manual

    print("\nSorted Unique Drivetrains:")
    for drivetrain in unique_drivetrains:
        print(drivetrain)
    # Doesn't need to be standardized, only a few values

    # Verify that fuel_type is always Gasoline
    print("\nSorted Unique Fuel Types:")
    for fuel_type in unique_fuel_types:
        print(fuel_type)
    # Only Gasoline, column can be removed.

### Too much output
#print_unqiues()

## Isolate important engine attributes: 
#### cylinders, liters, boosted

In [10]:
# Define regular expressions for extracting attributes
cylinders_pattern_1 = re.compile(r'(\d+)[\s-]*Cyl') # For cases like "6-Cyl" and "6 Cyl"
cylinders_pattern_2 = re.compile(r'(?:[IVHW]|VR)-?([3-9]|1[0-2])') # For cases like "V8"
liters_pattern_1 = re.compile(r'(\d+(?:\.\d+)?)L') # For cases like "3.6L"
liters_pattern_2 = re.compile(r'(\d+(?:\.\d+)?)\s*Liter') # For cases like "3.6 Liter"
boosted_pattern = re.compile(r'Turbo|Supercharge', re.IGNORECASE)

def extract_attributes(engine_str):
    cylinders_match_1 = re.search(cylinders_pattern_1, engine_str)
    cylinders_match_2 = re.search(cylinders_pattern_2, engine_str)
    if cylinders_match_1:
        cylinders = cylinders_match_1.group(1)
    elif cylinders_match_2:
        cylinders = cylinders_match_2.group(1) or cylinders_match_2.group(2)
    else:
        cylinders = None
    
    liters_match_1 = re.search(liters_pattern_1, engine_str)
    liters_match_2 = re.search(liters_pattern_2, engine_str)
    if liters_match_1:
        liters = liters_match_1.group(1)
    elif liters_match_2:
        liters = liters_match_2.group(1)
    else:
        liters = None
    # Handle the case where the liters are specified without 'L' like engine: "2.0"
    if not liters and engine_str.replace('.', '', 1).isdigit():
        liters = engine_str

    boosted = bool(re.search(boosted_pattern, engine_str))
    
    return cylinders, liters, boosted

# Apply the function to the 'engine' column
df[['Cylinders', 'Liters', 'Boosted']] = df['engine'].apply(extract_attributes).apply(pd.Series)

# Display the resulting DataFrame
#pd.reset_option('display.max_rows')
df[['engine', 'Cylinders', 'Liters', 'Boosted']].head()

,engine,Cylinders,Liters,Boosted
0,2.4L I4 16V GDI DOHC,4,2.4,False
1,5.7L V8 16V MPFI OHV,8,5.7,False
2,1.6L I4 16V MPFI DOHC,4,1.6,False
3,5.7L V8 16V MPFI OHV,8,5.7,False
4,3.6L V6 24V MPFI DOHC,6,3.6,False


In [11]:
### Check extracted engine attribute for unique engines
unique_engines = df['engine'].unique()
engine_data = []

for engine in unique_engines:
    cylinders, liters, boosted = extract_attributes(engine)
    engine_data.append({'Engine': engine, 'Cylinders': cylinders, 'Liters': liters, 'Boosted': boosted})

engine_df = pd.DataFrame(engine_data)

# 648 unique engines, too much to display.
#pd.set_option('display.max_rows', None)
#engine_df
#pd.reset_option('display.max_rows')

### Edge cases:
# row 295 "Electric"
# row 391 "-"

## Map color names to a basic set

In [45]:
color_df = df[['exterior_color', 'interior_color', 'link']].copy()

# Split listings with multiple colors 
color_df['exterior_color'] = color_df['exterior_color'].apply(lambda x: x.split(' / ') if pd.notnull(x) else x)
color_df['interior_color'] = color_df['interior_color'].apply(lambda x: x.split(' / ') if pd.notnull(x) else x)
color_df.head()

,exterior_color,interior_color,link
0,[Nightfall Blue],[Beige],"""https://www.cars.com/vehicledetail/5ee3ae04-6..."
1,[Silver],[Black],"""https://www.cars.com/vehicledetail/c99e5035-7..."
2,"[Monarch Orange, Super Black]",[Charcoal],"""https://www.cars.com/vehicledetail/4e79fa3a-b..."
3,[Gray],[Black],"""https://www.cars.com/vehicledetail/86a0d096-3..."
4,[Blue],[Black],"""https://www.cars.com/vehicledetail/36efbef5-f..."


In [46]:
# Define list of common colors (based on cars.com search)
common_colors = ['beige', 'black', 'blue', 'brown', 'gold', 'gray', 'green', 'orange', 'purple', 'red', 'silver', 'white', 'yellow']

def condense_colors(color_list):
    if color_list is not None:
        modified_colors = color_list

        # Missing or incorrect exterior color
        empty_conditions = [['-'], ['0'], [np.nan], ['Leather'], ['OTHER']]
        if modified_colors in empty_conditions:
            return []
        
        # Condense anything that contains a common color in the whole string to only that color
        modified_colors = [next((cc for cc in common_colors if cc in color.lower()), color) for color in modified_colors]

        # Replace colors variations with their common counterpart
        beige_variations = ['tan', 'khaki', 'kaikoura', 'Almond']
        for variation in beige_variations:
            modified_colors = [re.sub(fr'{re.escape(variation)}', 'beige', color, flags=re.IGNORECASE) for color in modified_colors]

        gold_variations = ['champagne']
        for variation in gold_variations:
            modified_colors = [re.sub(fr'{re.escape(variation)}', 'gold', color, flags=re.IGNORECASE) for color in modified_colors]

        black_variations = ['ebony', 'quartz', 'obsidian'] 
        # 'quartz' after 'champagne' because 'champagne quartz' is gold
        for variation in black_variations:
            modified_colors = [re.sub(fr'{re.escape(variation)}', 'black', color, flags=re.IGNORECASE) for color in modified_colors]
        
        blue_variations = ['sapphire', 'sea', 'marlin', 'cyan', 'aqua', 'blu', 'wave', 'indigo', 'ocean']
        for variation in blue_variations:
            modified_colors = [re.sub(fr'{re.escape(variation)}', 'blue', color, flags=re.IGNORECASE) for color in modified_colors]

        brown_variations = ['latte', 'cafe', 'mocha', 'chestnut', 'bronze', 'sandalwood']
        for variation in brown_variations:
            modified_colors = [re.sub(fr'\b{re.escape(variation)}\b', 'brown', color, flags=re.IGNORECASE) for color in modified_colors]

        red_variations = ['cherry', 'scarlet', 'rose', 'ruby', 'crimson', 'burgundy', 'maroon', 'lava']
        for variation in red_variations:
            modified_colors = [re.sub(fr'{re.escape(variation)}', 'red', color, flags=re.IGNORECASE) for color in modified_colors]

        gray_variations = ['grey', 'slate', 'graphite', 'storm', 'cement', 'cactus', 'steel', 'granite', 'smok', 'charcoal', 'gun', 'magnetic']
        # 'gun' after 'burgundy' because gun is in burgundy
        for variation in gray_variations:
            modified_colors = [re.sub(fr'{re.escape(variation)}', 'gray', color, flags=re.IGNORECASE) for color in modified_colors]

        green_variations = ['sage', 'woodland', 'lime', 'teal', 'cypress']
        for variation in green_variations:
            modified_colors = [re.sub(fr'{re.escape(variation)}', 'green', color, flags=re.IGNORECASE) for color in modified_colors]

        orange_variations = ['sunset', 'papaya', 'copper']
        for variation in orange_variations:
            modified_colors = [re.sub(fr'{re.escape(variation)}', 'orange', color, flags=re.IGNORECASE) for color in modified_colors]

        purple_variations = ['plum', 'amethyst', 'violet']
        for variation in purple_variations:
            modified_colors = [re.sub(fr'{re.escape(variation)}', 'purple', color, flags=re.IGNORECASE) for color in modified_colors]

        silver_variations = ['platinum', 'iridium', 'sliver']
        for variation in silver_variations:
            modified_colors = [re.sub(fr'{re.escape(variation)}', 'silver', color, flags=re.IGNORECASE) for color in modified_colors]

        white_variations = ['ivory', 'cream', 'bianco', 'ice', 'stone', 'iridescent', 'powder'] 
        # 'stone' after 'kaikoura' because 'kaikoura stone' is beige
        for variation in white_variations:
            modified_colors = [re.sub(fr'{re.escape(variation)}', 'white', color, flags=re.IGNORECASE) for color in modified_colors]

        yellow_variations = ['hellayella', 'giallo inti']
        for variation in yellow_variations:
            modified_colors = [re.sub(fr'{re.escape(variation)}', 'yellow', color, flags=re.IGNORECASE) for color in modified_colors]


        # After replacing variations, condense again
        modified_colors = [next((cc for cc in common_colors if cc in color.lower()), color) for color in modified_colors]

        return modified_colors
    else:
        return []

# Apply the condense_colors function to the 'exterior_color' column
color_df['exterior_color'] = color_df['exterior_color'].apply(condense_colors)

In [49]:
### Print remaining unique exterior colors with occurrences
def print_unique_exterior_colors():
    unique_exterior_colors = color_df['exterior_color'].explode()
    unique_colors_counts = unique_exterior_colors.value_counts()

    total_occurrences = unique_colors_counts[~unique_colors_counts.index.isin(common_colors)].sum()
    print(f"Total remaining: {total_occurrences} \n")

    for color, count in unique_colors_counts.items():
        if color not in common_colors:
            print(f"{color}: {count} occurrences")

print_unique_exterior_colors()


Total remaining: 176
Metallic: 8 occurrences
Metal: 7 occurrences
Starfire Pearl: 6 occurrences
Polished Metal Metallic: 6 occurrences
Blizzard Pearl: 5 occurrences
Avalanche: 5 occurrences
Caviar: 5 occurrences
Moonlight Cloud: 5 occurrences
Wind Chill Pearl: 5 occurrences
Shadow Metallic: 4 occurrences
Opulent Amber: 4 occurrences
Nightfall Mica: 3 occurrences
Night Armor: 3 occurrences
Snazzberry Pearlcoat: 3 occurrences
Dark Ash Metallic: 3 occurrences
Area 51: 3 occurrences
Lunar Rock: 3 occurrences
Tungsten Metallic: 3 occurrences
Argento Nurburgring Metallic: 3 occurrences
Mojito! Clearcoat: 2 occurrences
Aruba: 2 occurrences
Mercury Metallic: 2 occurrences
Marine Metallic: 2 occurrences
Midnight Garnet: 2 occurrences
Agate: 2 occurrences
Gobi Clearcoat: 2 occurrences
Billet Clearcoat: 2 occurrences
Chocolate Metallic: 2 occurrences
Pearl: 2 occurrences
Deep: 2 occurrences
Lead Foot: 2 occurrences
Cinnabar Metallic: 1 occurrences
Rocky Mountain Pearlcoat: 1 occurrences
Havana Me

In [28]:
### Check the maximum number of colors listed together
max_exterior_color_length = color_df['exterior_color'].apply(len).max()
max_interior_color_length = color_df['interior_color'].apply(len).max()
print(f"Max Exterior Color List Length: {max_exterior_color_length}")
print(f"Max Interior Color List Length: {max_interior_color_length}")

### After checking the 8 colors that have more than 2 listed, 
### I can see that 2 color columns for each exterior and interior will be sufficient.

Max Exterior Color List Length: 2
Max Interior Color List Length: 4


In [37]:
#pd.set_option('display.max_rows', None)
#color_df
#pd.reset_option('display.max_rows')

## Adjusting MPG to High MPG, Low MPG

In [8]:
def clean_mpg(mpg):
    # Takes mpg string from webscrape and turns it to [low, high]
    # input options are nan, -, 23, 23-25
    mpg = str(mpg)
    if mpg == np.nan: # handles nan
        return [np.nan, np.nan]
    elif mpg == '-': # handles -
        return [np.nan, np.nan]
    else:  # handles actual mpgs
        return mpg.split('-')

df['low mpg'] = df.mpg.apply(lambda x: clean_mpg(x)[0]) # low number always first
df['high mpg'] = df.mpg.apply(lambda x: clean_mpg(x)[-1]) # high number always last, if only one number its both and list len = 1
df['low mpg'] = pd.to_numeric(df['low mpg'], errors='coerce').fillna(-1).astype(int)
df['high mpg'] = pd.to_numeric(df['high mpg'], errors='coerce').fillna(-1).astype(int)
df['high mpg'] = df['high mpg'].fillna(-1).astype(int)
df['low mpg'] = df['low mpg'].replace(-1, np.nan)
df['high mpg'] = df['high mpg'].replace(-1, np.nan)


array([21., 16., 31., 19., 14., 23., 22., 25., 26., 24., 28., 20., 27.,
       13., 17., 18., nan, 29., 15., 30., 11., 12., 51., 10., 33., 39.,
       55.,  0., 32., 76., 41., 38., 34., 42., 80., 40., 36., 35., 43.])

In [17]:
df.shape

(6002, 31)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6002 entries, 0 to 6001
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   link                     6002 non-null   object        
 1   listing_title            6002 non-null   object        
 2   listing_mileage          6002 non-null   int32         
 3   primary_price            6002 non-null   int32         
 4   deal_gauge               5524 non-null   object        
 5   exterior_color           6002 non-null   object        
 6   interior_color           6002 non-null   object        
 7   drivetrain               6002 non-null   object        
 8   mpg                      5957 non-null   object        
 9   fuel_type                6002 non-null   object        
 10  transmission             6002 non-null   object        
 11  engine                   6002 non-null   object        
 12  vin                      6002 non-

In [19]:
df.to_csv('new_cars.csv',index=False)